# Basic BVAS demo using simulated data

In [3]:
from bvas import simulate_data, BVASSelector

### Simulate data

In [5]:
data = simulate_data(num_alleles=100, duration=26, num_variants=100, num_regions=10,
                     k=0.1, seed=0, sampling_rate=10, strategy='global-median')

In [35]:
# inspect simulated data
for k, v in data.items():
    print(k, v.shape)
    
print("\nEstimated effective population size: {:.1f}".format(data['estimated_nu_eff'].item()))

Y torch.Size([100])
Gamma torch.Size([100, 100])
estimated_nu_eff (1,)
true_betas torch.Size([100])

Estimated effective population size: 490.3


### Instantiate BVASSelector object

In [18]:
# create names for our 100 alleles (the first 10 alleles are non-neutral in the simulation)
mutations = ["Causal{}".format(k) for k in range(1, 11)] 
mutations += ["Spurious{}".format(k) for k in range(11, 101)] 

selector = BVASSelector(data['Y'], 
                        data['Gamma'], 
                        mutations, 
                        S=5.0,
                        tau=100.0)

### Run BVAS MCMC-based inference

In [19]:
selector.run(T=1000, T_burnin=500)

  0%|          | 0/1500 [00:00<?, ?it/s]

### Inspect results

- We find that 8 of the 10 true causal alleles are assigned large PIPs
- We find that 2 of the 10 true causal alleles are missed (i.e. those with small effect sizes, namely Causal1 and Causal6)
- We find that no spurious alleles are assigned large PIPs
- We see that the beta estimates are regularized somewhat towards zero

In [36]:
print(selector.summary.iloc[:15][['PIP', 'Beta', 'BetaStd', 'Rank']])

                 PIP      Beta   BetaStd  Rank
Causal4     0.999999  0.051166  0.006216     1
Causal5     0.999999  0.066024  0.006334     2
Causal10    0.999999 -0.068597  0.008873     3
Causal9     0.999999 -0.065844  0.010435     4
Causal3     0.999903  0.039212  0.006989     5
Causal8     0.881331 -0.026670  0.013342     6
Causal7     0.250416 -0.005248  0.010489     7
Causal2     0.133008  0.003241  0.008441     8
Spurious89  0.023355 -0.000385  0.002696     9
Spurious24  0.018575  0.000173  0.001668    10
Spurious70  0.016120  0.000370  0.002402    11
Spurious80  0.013704  0.000187  0.001622    12
Spurious21  0.011855  0.000109  0.001287    13
Spurious16  0.009838 -0.000098  0.001316    14
Spurious44  0.009189 -0.000078  0.001053    15


In [32]:
# print true betas for the causal coefficients
for mutation, beta in zip(mutations[:10], data['true_betas'][:10]):
    print("[{}]\t{:.2f}".format(mutation, beta.item()))

[Causal1]	0.01
[Causal2]	0.02
[Causal3]	0.04
[Causal4]	0.06
[Causal5]	0.08
[Causal6]	-0.01
[Causal7]	-0.02
[Causal8]	-0.04
[Causal9]	-0.06
[Causal10]	-0.08
